In [14]:
import pickle
import numpy as np
from scipy.spatial import distance

In [3]:
# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [6]:
#create dictionary with words a labels and the EEG embeddings in a list as the values

EEG_word_level_dict = {}
for i in range(len(EEG_word_level_labels)):
    if EEG_word_level_labels[i] in EEG_word_level_dict:
        EEG_word_level_dict[EEG_word_level_labels[i]].append(EEG_word_level_embeddings[i])
    else:
        EEG_word_level_dict[EEG_word_level_labels[i]] = [EEG_word_level_embeddings[i]]


In [24]:
def average_eeg_segments(eeg_segments):
    """
    Compute the average EEG segment from a list of EEG segments.

    Parameters:
        eeg_segments (list of array-like): List of EEG segment data.

    Returns:
        array-like: Average EEG segment.
    """
    # Stack EEG segments along a new axis to compute the average
    stacked_segments = np.stack(eeg_segments, axis=0)

    # Compute the mean across segments
    avg_segment = np.mean(stacked_segments, axis=0)

    return avg_segment

# Example usage
# Assuming eeg_segments_dict is a dictionary where keys are words and values are lists of EEG segments

# Dictionary containing average EEG segment for each word
average_segments_dict = {}

for word, segments in EEG_word_level_dict.items():
    # Compute average EEG segment for the current word
    avg_segment = average_eeg_segments(segments)

    # Store average segment in dictionary
    average_segments_dict[word] = avg_segment

In [27]:
def convert_to_probability_distribution(eeg_segment):
    """
    Convert EEG segment to a probability distribution.

    Parameters:
        eeg_segment (array-like): EEG segment data.

    Returns:
        array-like: Probability distribution representing the EEG segment.
    """
    # Resize EEG segment to a 1D array
    flattened_segment = eeg_segment.ravel()

    # Normalize the flattened segment
    normalized_segment = (flattened_segment - np.mean(flattened_segment)) / np.std(flattened_segment)

    # Convert normalized segment into probability values
    # For example, you can apply softmax function
    probabilities = np.exp(normalized_segment) / np.sum(np.exp(normalized_segment))

    return probabilities

In [28]:
convert_to_probability_distribution(average_segments_dict['the'])

array([2.39234113e-04, 3.81966436e-04, 4.66857337e-04, 5.30187161e-04,
       4.57271920e-04, 2.58426682e-04, 2.53501715e-04, 4.11752738e-04,
       4.56164551e-04, 5.86777769e-04, 4.29269778e-04, 2.63610116e-04,
       3.80216303e-04, 3.82986626e-04, 4.80952701e-04, 4.81583049e-04,
       2.74355674e-04, 3.78774434e-04, 4.49914052e-04, 2.45695786e-04,
       3.84282832e-04, 4.63564528e-04, 4.61969565e-04, 3.96568189e-04,
       2.51863955e-04, 3.01488240e-04, 4.15266241e-04, 4.95071588e-04,
       5.47881923e-04, 4.97678024e-04, 2.13003583e-04, 5.70976392e-04,
       6.55395702e-04, 7.28570437e-04, 8.79134724e-04, 3.09554405e-04,
       4.66356544e-04, 1.20312752e-03, 1.17041692e-03, 1.12652085e-03,
       2.22969299e-03, 1.76340376e-03, 1.42173184e-03, 1.04045424e-03,
       6.14542191e-04, 2.68376047e-04, 2.64811297e-03, 3.45209316e-03,
       2.72339504e-03, 1.95409931e-03, 1.27833247e-03, 1.95004947e-03,
       4.93258141e-03, 5.68783462e-03, 3.81977496e-03, 2.77919495e-03,
      

In [29]:
def compute_js_distance(p, q):
    """
    Compute Jensen-Shannon distance between two probability distributions.

    Parameters:
        p (array-like): Probability distribution.
        q (array-like): Probability distribution.

    Returns:
        float: Jensen-Shannon distance between distributions.
    """
    # Normalize distributions
    p = p / np.sum(p)
    q = q / np.sum(q)

    # Compute average distribution
    m = 0.5 * (p + q)

    # Compute Jensen-Shannon divergence
    js_divergence = 0.5 * (distance.jensenshannon(p, m) + distance.jensenshannon(q, m))

    return js_divergence

In [32]:
value = convert_to_probability_distribution(average_segments_dict['the'])
value_2 = convert_to_probability_distribution(average_segments_dict['and'])

In [33]:
compute_js_distance(value, value_2)

0.0082674372719081

In [56]:


mean_eeg = np.mean(average_segments_dict['and'])
std_dev_eeg = np.std(average_segments_dict['and'])

value_3 = np.random.normal(loc=mean_eeg, scale=std_dev_eeg, size=average_segments_dict['and'].shape)

In [58]:
value_3 = convert_to_probability_distribution(value_3)

In [59]:
compute_js_distance(value_2, value_3)

0.2390583352673319